# Valence value regression based on Deap Dataset

## 0. This notebook is based on DEAP database

Anyone should refer to DEAP team first

@article{koelstra2012deap,
  title={Deap: A database for emotion analysis; using physiological signals},
  author={Koelstra, Sander and Muhl, Christian and Soleymani, Mohammad and Lee, Jong-Seok and Yazdani, Ashkan and Ebrahimi, Touradj and Pun, Thierry and Nijholt, Anton and Patras, Ioannis},
  journal={IEEE Transactions on Affective Computing},
  volume={3},
  number={1},
  pages={18--31},
  year={2012},
  publisher={IEEE}
}

## 1. Dependency
* numpy
* pyEEG
* sciki-learn

In [ ]:
!pip install numpy
!pip install pandas

!pip install scikit-learn


!pip install tensorflow

In [1]:
!pip install git+https://github.com/forrestbao/pyeeg.git


  Cloning https://github.com/forrestbao/pyeeg.git to /tmp/pip-req-build-41cz_n5h
  Running command git clone --filter=blob:none --quiet https://github.com/forrestbao/pyeeg.git /tmp/pip-req-build-41cz_n5h
  Resolved https://github.com/forrestbao/pyeeg.git to commit a6c18bb093e4748f9d9c208535a6ae024a0802b8
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install tensorflow

In [2]:
import numpy as np
import pyeeg as pe
import pickle as pickle
import pandas as pd
import math

from sklearn import svm
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

import os
import tensorflow as tf
import time

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
!pip install PyWavelets

  Using cached pywavelets-1.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.0 kB)
Using cached pywavelets-1.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.5 MB)


## 2. Global Variables setup
File Name data\SXX.dat, XX \in [0,31]
* data: 40 x 40 x 8064: trial x channel x data
* label: 40 x 4: video/trial x label (valence, arousal, dominance, liking)

Channel Indice: {
* 1 : AF3; 2: F3; 3: F7; 4: FC5; 7: T7; 11: P7; 13: O1
* 17: AF4; 19: F4; 20: F8; 21: FC6; 25: T8; 29: P8; 31: O2 }

In [5]:
eeg_channels = np.array(["Fp1", "AF3", "F3", "F7", "FC5", "FC1", "C3", "T7", "CP5", "CP1", "P3", "P7", "PO3", "O1", "Oz", "Pz", "Fp2", "AF4", "Fz", "F4", "F8", "FC6", "FC2", "Cz", "C4", "T8", "CP6", "CP2", "P4", "P8", "PO4", "O2"])
channel_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
peripheral_channels = np.array(["hEOG", "vEOG", "zEMG", "tEMG", "GSR", "Respiration belt", "Plethysmograph", "Temperature"])
band = [4,8,12,30,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']
#List of subjects

## 3. FFT with pyeeg
* [4-8]: theta band
* [8-12]: alpha band
* [12-30]: beta band 
* [30-45]: low gamma band
* 

In [8]:
from pywt import wavedec
def DWT_Processing (sub, channel_indices, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = list([])
    with open('../data/s' + sub + '.dat', 'rb') as file:
        
        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1

        for i in range (0,40):
            # loop over 0-39 trails
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0;

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel_indices:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    coeffs = wavedec(X, 'sym9',level=5,mode='symmetric')
                    Y = [np.sum(np.square(coeffs[4])),np.sum(np.square(coeffs[3])),np.sum(np.square(coeffs[2])),np.sum(np.square(coeffs[1]))]
                    
                    
                    #meta_data = meta_data + list(Y[0])
                    meta_data.extend(Y)  # Append the power values

                meta_array = list([np.array(meta_data), np.array(labels)])  # Create a homogeneous list
                meta.append(meta_array) 

                #meta_array.append(np.array(meta_data))
                #meta_array.append(labels)

                #meta.append(np.array(meta_array))

                start = start + step_size
                
    meta = np.array(meta,dtype=object)
    np.save('dwt/out\s' + sub + str(window_size), meta, allow_pickle=True, fix_imports=True)

def testing (M, L, model):
    '''
    arguments:  M: testing dataset
                L: testing dataset label
                model: scikit-learn model

    return:     void
    '''
    output = model.predict(M[0:78080:32])
    label = L[0:78080:32]

    k = 0
    l = 0

    for i in range(len(label)):
        k = k + (output[i] - label[i])*(output[i] - label[i]) #square difference 

        #a good guess
        if (output[i] > 5 and label[i] > 5):
            l = l + 1
        elif (output[i] < 5 and label[i] <5):
            l = l + 1

    print ("l2 error:", k/len(label), "classification accuracy:", l / len(label),l, len(label))

In [8]:
window_size = 128
step_size = 64
for subjects in subjectList:
    DWT_Processing (subjects, channel_indices, band, window_size, step_size, sample_rate)


/opt/conda/lib/python3.10/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 5 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [9]:
window_size = 256
step_size = 128
for subjects in subjectList:
    DWT_Processing (subjects, channel_indices, band, window_size, step_size, sample_rate)


## 3.Segment of preprocessed data
* training dataset: 80 %
 *testing dataset: 20%

Agrithom pool:
* Support Vector Machine (which kernal?)
* knn
*RF

Best practice could be refered to this paper: 

@article{alarcao2017emotions,
  title={Emotions recognition using EEG signals: A survey},
  author={Alarcao, Soraia M and Fonseca, Manuel J},
  journal={IEEE Transactions on Affective Computing},
  year={2017},
  publisher={IEEE}
}

In [10]:
#for subjects in subjectList:
data_training_2s = []
label_training_2s = []
data_testing_2s = []
label_testing_2s = []
data_all = []
labels_all = []
window_size = 128

for subjects in subjectList:
    with open('dwt/out\s' + subjects + str(window_size) + '.npy', 'rb') as file:
        sub = np.load(file, allow_pickle=True)
        data = [item[0] for item in sub]
        labels = [item[1] for item in sub]
        data_all.extend(data)
        labels_all.extend(labels)

# Convert to numpy arrays for consistency
data_all = np.array(data_all)
labels_all = np.array(labels_all)
        
        # First split: train+val and test (80% train+val, 20% test)
data_training_2s, data_testing_2s, label_training_2s, label_testing_2s = train_test_split(data_all, labels_all, test_size=0.2, random_state=42)
       
       
np.save('out\dwt_data_training_2s', np.array(data_training_2s), allow_pickle=True, fix_imports=True)
np.save('out\dwt_label_training_2s', np.array(label_training_2s), allow_pickle=True, fix_imports=True)
print("training dataset_2s:", np.array(data_training_2s).shape, np.array(label_training_2s).shape)

np.save('out\dwt_data_testing_2s', np.array(data_testing_2s), allow_pickle=True, fix_imports=True)
np.save('out\dwt_label_testing_2s', np.array(label_testing_2s), allow_pickle=True, fix_imports=True)
print("testing dataset_2s:", np.array(data_testing_2s).shape, np.array(label_testing_2s).shape)

training dataset_2s: (126976, 128) (126976, 4)
testing dataset_2s: (31744, 128) (31744, 4)


In [11]:
window_size = 640
step_size = 320
for subjects in subjectList:
    DWT_Processing (subjects, channel_indices, band, window_size, step_size, sample_rate)
window_size = 1280
step_size = 640
for subjects in subjectList:
    DWT_Processing (subjects, channel_indices, band, window_size, step_size, sample_rate)

In [12]:
#for subjects in subjectList:
data_training_5s = []
label_training_5s = []
data_testing_5s = []
label_testing_5s = []
data_all = []
labels_all = []
window_size = 640

for subjects in subjectList:
   
    with open('dwt/out\s' + subjects + str(window_size) + '.npy', 'rb') as file:
        sub = np.load(file, allow_pickle=True)
        data = [item[0] for item in sub]
        labels = [item[1] for item in sub]
        data_all.extend(data)
        labels_all.extend(labels)

# Convert to numpy arrays for consistency
data_all = np.array(data_all)
labels_all = np.array(labels_all)
        
        # First split: train+val and test (80% train+val, 20% test)
data_training_5s, data_testing_5s, label_training_5s, label_testing_5s = train_test_split(data_all, labels_all, test_size=0.2, random_state=42)
       


np.save('out\dwt_data_training_5s', np.array(data_training_5s), allow_pickle=True, fix_imports=True)
np.save('out\dwt_label_training_5s', np.array(label_training_5s), allow_pickle=True, fix_imports=True)
print("training dataset_5s:", np.array(data_training_5s).shape, np.array(label_training_5s).shape)

np.save('out\dwt_data_testing_5s', np.array(data_testing_5s), allow_pickle=True, fix_imports=True)
np.save('out\dwt_label_testing_5s', np.array(label_testing_5s), allow_pickle=True, fix_imports=True)
print("testing datase_5st:", np.array(data_testing_5s).shape, np.array(label_testing_5s).shape)


training dataset_5s: (24576, 128) (24576, 4)
testing datase_5st: (6144, 128) (6144, 4)


In [13]:
#for subjects in subjectList:
data_training_10s = []
label_training_10s = []
data_testing_10s = []
label_testing_10s = []
data_all = []
labels_all = []
window_size = 1280

for subjects in subjectList:

     with open('dwt/out\s' + subjects + str(window_size) + '.npy', 'rb') as file:
        sub = np.load(file, allow_pickle=True)
        data = [item[0] for item in sub]
        labels = [item[1] for item in sub]
        data_all.extend(data)
        labels_all.extend(labels)

# Convert to numpy arrays for consistency
data_all = np.array(data_all)
labels_all = np.array(labels_all)
        
        # First split: train+val and test (80% train+val, 20% test)
data_training_10s, data_testing_10s, label_training_10s, label_testing_10s = train_test_split(data_all, labels_all, test_size=0.2, random_state=42)
       

np.save('out\dwt_data_training_10s', np.array(data_training_10s), allow_pickle=True, fix_imports=True)
np.save('out\dwt_label_training_10s', np.array(label_training_10s), allow_pickle=True, fix_imports=True)
print("training dataset_10s:", np.array(data_training_10s).shape, np.array(label_training_10s).shape)

np.save('out\dwt_data_testing_10s', np.array(data_testing_10s), allow_pickle=True, fix_imports=True)
np.save('out\dwt_label_testing_10s', np.array(label_testing_10s), allow_pickle=True, fix_imports=True)
print("testing dataset_10s:", np.array(data_testing_10s).shape, np.array(label_testing_10s).shape)



training dataset_10s: (11264, 128) (11264, 4)
testing dataset_10s: (2816, 128) (2816, 4)


In [17]:
#for subjects in subjectList:
data_training_1s = []
label_training_1s = []
data_testing_1s = []
label_testing_1s = []
data_all = []
labels_all = []
window_size = 256

for subjects in subjectList:
    with open('dwt/out\s' + subjects + str(window_size) + '.npy', 'rb') as file:
        sub = np.load(file, allow_pickle=True)
        data = [item[0] for item in sub]
        labels = [item[1] for item in sub]
        data_all.extend(data)
        labels_all.extend(labels)

# Convert to numpy arrays for consistency
data_all = np.array(data_all)
labels_all = np.array(labels_all)
        
        # First split: train+val and test (80% train+val, 20% test)
data_training_1s, data_testing_1s, label_training_1s, label_testing_1s = train_test_split(data_all, labels_all, test_size=0.2, random_state=42)
       
       
np.save('out\dwt_data_training_1s', np.array(data_training_1s), allow_pickle=True, fix_imports=True)
np.save('out\dwt_label_training_1s', np.array(label_training_1s), allow_pickle=True, fix_imports=True)
print("training dataset_1s:", np.array(data_training_1s).shape, np.array(label_training_1s).shape)

np.save('out\dwt_data_testing_1s', np.array(data_testing_1s), allow_pickle=True, fix_imports=True)
np.save('out\dwt_label_testing_1s', np.array(label_testing_1s), allow_pickle=True, fix_imports=True)
print("testing dataset_1s:", np.array(data_testing_1s).shape, np.array(label_testing_1s).shape)

training dataset_1s: (62464, 128) (62464, 4)
testing dataset_1s: (15616, 128) (15616, 4)


## 4.Regression
### 0. Loading Training and Testing dataset

In [14]:
with open('out\dwt_data_training_2s.npy', 'rb') as fileTrain:
    X_2s  = np.load(fileTrain)
    
with open('out\dwt_label_training_2s.npy', 'rb') as fileTrainL:
    Y_2s  = np.load(fileTrainL)

ss = StandardScaler()
X_2s = ss.fit_transform(X_2s)
Z_2s = np.ravel(Y_2s[:, [1]])

Arousal_Train_2s = np.ravel(Y_2s[:, [0]])
Valence_Train_2s = np.ravel(Y_2s[:, [1]])
Domain_Train_2s = np.ravel(Y_2s[:, [2]])
Like_Train_2s = np.ravel(Y_2s[:, [3]])



with open('out\dwt_data_testing_2s.npy', 'rb') as fileTrain:
    M_2s  = np.load(fileTrain)
    
with open('out\dwt_label_testing_2s.npy', 'rb') as fileTrainL:
    N_2s  = np.load(fileTrainL)

M_2s = ss.transform(M_2s)
L_2s = np.ravel(N_2s[:, [1]])

Arousal_Test_2s = np.ravel(N_2s[:, [0]])
Valence_Test_2s = np.ravel(N_2s[:, [1]])
Domain_Test_2s = np.ravel(N_2s[:, [2]])
Like_Test_2s = np.ravel(N_2s[:, [3]])


### 1. Support Vector Regression
* default setting, l1 error: 1.621761042477756 classification error: 0.6057377049180328 1478 2440

In [15]:
import numpy as np

# Creating the labels for training data
Y_train_2s = np.zeros_like(Arousal_Train_2s)
condition1_train = (Arousal_Train_2s >= 5) & (Valence_Train_2s >= 5)
condition2_train = (Arousal_Train_2s >= 5) & (Valence_Train_2s < 5)
condition3_train = (Arousal_Train_2s < 5) & (Valence_Train_2s >= 5)
condition4_train = (Arousal_Train_2s < 5) & (Valence_Train_2s < 5)

Y_train_2s[condition1_train] = 0
Y_train_2s[condition2_train] = 1
Y_train_2s[condition3_train] = 2
Y_train_2s[condition4_train] = 3

# Creating the labels for testing data
Y_test_2s = np.zeros_like(Arousal_Test_2s)
condition1_test = (Arousal_Test_2s >= 5) & (Valence_Test_2s >= 5)
condition2_test = (Arousal_Test_2s >= 5) & (Valence_Test_2s < 5)
condition3_test = (Arousal_Test_2s < 5) & (Valence_Test_2s >= 5)
condition4_test = (Arousal_Test_2s < 5) & (Valence_Test_2s < 5)

Y_test_2s[condition1_test] = 0
Y_test_2s[condition2_test] = 1
Y_test_2s[condition3_test] = 2
Y_test_2s[condition4_test] = 3

print("Y_train_2s:", np.unique(Y_train_2s, return_counts=True))
print("Y_test_2s:", np.unique(Y_test_2s, return_counts=True))


Y_train_2s: (array([0., 1., 2., 3.]), array([22365, 13012, 14446, 12641]))
Y_test_2s: (array([0., 1., 2., 3.]), array([5573, 3214, 3610, 3219]))


In [16]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report

param_grid = {
    'C': [0.1, 1, 10],  # Regularization parameter
    'kernel': ['linear', 'rbf']  # Type of kernel
}

# Setup SVM classifier
svm = SVC(probability=True, decision_function_shape='ovo')  # 'ovo' for one-vs-one

# Setup Grid Search for SVM
grid_search = GridSearchCV(
    svm, param_grid, cv=5, scoring='f1_weighted', n_jobs=-1
)

# Fit Grid Search
grid_search.fit(X_2s, Y_train_2s)

# Best parameters
print("Best parameters:", grid_search.best_params_)

# Predict and evaluate
y_pred_2s = grid_search.predict(M_2s)

# Evaluation metrics
accuracy = accuracy_score(Y_test_2s, y_pred_2s)
f1 = f1_score(Y_test_2s, y_pred_2s, average='weighted')

print("Accuracy:", accuracy)
print("F1 score:", f1)

KeyboardInterrupt: 

In [ ]:
with open('out\data_training_5s.npy', 'rb') as fileTrain:
    X_5s  = np.load(fileTrain)
    
with open('out\label_training_5s.npy', 'rb') as fileTrainL:
    Y_5s  = np.load(fileTrainL)

ss = StandardScaler()
X_5s = ss.fit_transform(X_5s)
Z_5s = np.ravel(Y_5s[:, [1]])

Arousal_Train_5s = np.ravel(Y_5s[:, [0]])
Valence_Train_5s = np.ravel(Y_5s[:, [1]])
Domain_Train_5s = np.ravel(Y_5s[:, [2]])
Like_Train_5s = np.ravel(Y_5s[:, [3]])



with open('out\data_testing_5s.npy', 'rb') as fileTrain:
    M_5s  = np.load(fileTrain)
    
with open('out\label_testing_5s.npy', 'rb') as fileTrainL:
    N_5s  = np.load(fileTrainL)

M_5s = ss.transform(M_5s)
L_5s = np.ravel(N_5s[:, [1]])

Arousal_Test_5s = np.ravel(N_5s[:, [0]])
Valence_Test_5s = np.ravel(N_5s[:, [1]])
Domain_Test_5s = np.ravel(N_5s[:, [2]])
Like_Test_5s = np.ravel(N_5s[:, [3]])

# Creating the labels for training data
Y_train_5s = np.zeros_like(Arousal_Train_5s)
condition1_train = (Arousal_Train_5s >= 5) & (Valence_Train_5s >= 5)
condition2_train = (Arousal_Train_5s >= 5) & (Valence_Train_5s < 5)
condition3_train = (Arousal_Train_5s < 5) & (Valence_Train_5s >= 5)
condition4_train = (Arousal_Train_5s < 5) & (Valence_Train_5s < 5)

Y_train_5s[condition1_train] = 0
Y_train_5s[condition2_train] = 1
Y_train_5s[condition3_train] = 2
Y_train_5s[condition4_train] = 3

# Creating the labels for testing data
Y_test_5s = np.zeros_like(Arousal_Test_5s)
condition1_test = (Arousal_Test_5s >= 5) & (Valence_Test_5s >= 5)
condition2_test = (Arousal_Test_5s >= 5) & (Valence_Test_5s < 5)
condition3_test = (Arousal_Test_5s < 5) & (Valence_Test_5s >= 5)
condition4_test = (Arousal_Test_5s < 5) & (Valence_Test_5s < 5)

Y_test_5s[condition1_test] = 0
Y_test_5s[condition2_test] = 1
Y_test_5s[condition3_test] = 2
Y_test_5s[condition4_test] = 3

print("Y_train_5s:", np.unique(Y_train_5s, return_counts=True))
print("Y_test_5s:", np.unique(Y_test_5s, return_counts=True))

param_grid = {
    'C': [0.1, 1, 10],  # Regularization parameter
    'kernel': ['linear', 'rbf']  # Type of kernel
}

# Setup SVM classifier
svm = SVC(probability=True, decision_function_shape='ovo')  # 'ovo' for one-vs-one

# Setup Grid Search for SVM
grid_search = GridSearchCV(
    svm, param_grid, cv=5, scoring='f1_weighted', n_jobs=-1
)

# Fit Grid Search
grid_search.fit(X_5s, Y_train_5s)

# Best parameters
print("Best parameters:", grid_search.best_params_)

# Predict and evaluate
y_pred_5s = grid_search.predict(M_5s)

# Evaluation metrics
accuracy = accuracy_score(Y_test_5s, y_pred_5s)
f1 = f1_score(Y_test_5s, y_pred_5s, average='weighted')

print("Accuracy:", accuracy)
print("F1 score:", f1)


In [ ]:
with open('out\data_training_10s.npy', 'rb') as fileTrain:
    X_10s  = np.load(fileTrain)
    
with open('out\label_training_10s.npy', 'rb') as fileTrainL:
    Y_10s  = np.load(fileTrainL)

ss = StandardScaler()
X_10s = ss.fit_transform(X_10s)
Z_10s = np.ravel(Y_10s[:, [1]])

Arousal_Train_10s = np.ravel(Y_10s[:, [0]])
Valence_Train_10s = np.ravel(Y_10s[:, [1]])
Domain_Train_10s = np.ravel(Y_10s[:, [2]])
Like_Train_10s = np.ravel(Y_10s[:, [3]])

with open('out\data_testing_10s.npy', 'rb') as fileTrain:
    M_10s  = np.load(fileTrain)
    
with open('out\label_testing_10s.npy', 'rb') as fileTrainL:
    N_10s  = np.load(fileTrainL)

M_10s = ss.transform(M_10s)
L_10s = np.ravel(N_10s[:, [1]])

Arousal_Test_10s = np.ravel(N_10s[:, [0]])
Valence_Test_10s = np.ravel(N_10s[:, [1]])
Domain_Test_10s = np.ravel(N_10s[:, [2]])
Like_Test_10s = np.ravel(N_10s[:, [3]])

# Creating the labels for training data
Y_train_10s = np.zeros_like(Arousal_Train_10s)
condition1_train = (Arousal_Train_10s >= 5) & (Valence_Train_10s >= 5)
condition2_train = (Arousal_Train_10s >= 5) & (Valence_Train_10s < 5)
condition3_train = (Arousal_Train_10s < 5) & (Valence_Train_10s >= 5)
condition4_train = (Arousal_Train_10s < 5) & (Valence_Train_10s < 5)

Y_train_10s[condition1_train] = 0
Y_train_10s[condition2_train] = 1
Y_train_10s[condition3_train] = 2
Y_train_10s[condition4_train] = 3

# Creating the labels for testing data
Y_test_10s = np.zeros_like(Arousal_Test_5s)
condition1_test = (Arousal_Test_10s >= 5) & (Valence_Test_10s >= 5)
condition2_test = (Arousal_Test_10s >= 5) & (Valence_Test_10s < 5)
condition3_test = (Arousal_Test_10s < 5) & (Valence_Test_10s >= 5)
condition4_test = (Arousal_Test_10s < 5) & (Valence_Test_10s < 5)

Y_test_10s[condition1_test] = 0
Y_test_10s[condition2_test] = 1
Y_test_10s[condition3_test] = 2
Y_test_10s[condition4_test] = 3

print("Y_train_10s:", np.unique(Y_train_10s, return_counts=True))
print("Y_test_10s:", np.unique(Y_test_10s, return_counts=True))

param_grid = {
    'C': [0.1, 1, 10],  # Regularization parameter
    'kernel': ['linear', 'rbf']  # Type of kernel
}

# Setup SVM classifier
svm = SVC(probability=True, decision_function_shape='ovo')  # 'ovo' for one-vs-one

# Setup Grid Search for SVM
grid_search = GridSearchCV(
    svm, param_grid, cv=5, scoring='f1_weighted', n_jobs=-1
)

# Fit Grid Search
grid_search.fit(X_10s, Y_train_10s)

# Best parameters
print("Best parameters:", grid_search.best_params_)

# Predict and evaluate
y_pred_10s = grid_search.predict(M_10s)

# Evaluation metrics
accuracy = accuracy_score(Y_test_10s, y_pred_10s)
f1 = f1_score(Y_test_10s, y_pred_10s, average='weighted')

print("Accuracy:", accuracy)
print("F1 score:", f1)